In [ ]:
import warnings
import pandas as pd
from draco import dict_union, dict_to_facts, Draco
from draco.renderer import AltairRenderer
from IPython.display import display
import json

warnings.filterwarnings("ignore")

dataset = pd.read_csv("Superstore.csv", encoding="ISO-8859-1")
datasetFiltered = dataset[dataset[['Rowid',"Orderid","Orderdate","Shipdate","Shipmode","Customerid","Customername","Segment","Country","City","State","Postalcode","Region","Productid","Category","Subcategory","Productname","Sales","Quantity","Discount","Profit"]].notnull().all(1)]

datasetFiltered["Orderdate"] = pd.to_datetime(datasetFiltered["Orderdate"])
print(datasetFiltered['Orderdate'].dtype)
datasetFiltered['MonthYear'] = pd.to_datetime(datasetFiltered['Orderdate']).dt.strftime('%Y-%m')
datasetFiltered['Month'] = datasetFiltered["Orderdate"].dt.month_name().str.slice(stop=3)
print(datasetFiltered['MonthYear'])



In [ ]:
datasetFiltered.head()

In [ ]:
d = Draco()
renderer = AltairRenderer(concat_mode="hconcat")


def show(*args, df: pd.DataFrame = datasetFiltered):
    spec = dict_union(*args)
    prog = dict_to_facts(spec)
    if not d.check_spec(prog):
        print("\n".join(prog))
        print(d.get_violations(prog))
        assert False, "Invalid spec"

    # Display the rendered VL chart and the ASP
    chart = renderer.render(spec, df)
    print(json.dumps(spec, indent=2))
    display(chart)
    display(prog)

def data(fields: list[str], df: pd.DataFrame = datasetFiltered) -> dict:
    number_rows, _ = df.shape
    return {
        "number_rows": number_rows,
        "field": [
            x
            for x in [
                {"name": "Rowid", "type": "string", "__id__": "Rowid"},
                {"name": "Orderid", "type": "string", "__id__": "Orderid"},
                {"name": "Orderdate", "type": "datetime", "__id__": "Orderdate"},
                {"name": "Shipdate", "type": "datetime", "__id__": "Shipdate"},
                {"name": "Shipmode", "type": "string", "__id__": "Shipmode"},
                {"name": "Customerid", "type": "string", "__id__": "Customerid"},
                {"name": "Customername", "type": "string", "__id__": "Customername"},
                {"name": "Segment", "type": "string", "__id__": "Segment"},
                {"name": "Country", "type": "string", "__id__": "Country"},
                {"name": "City", "type": "string", "__id__": "City"},
                {"name": "State", "type": "string", "__id__": "State"},
                {"name": "Postalcode", "type": "number", "__id__": "Postalcode"},
                {"name": "Region", "type": "string", "__id__": "Region"},
                {"name": "Productid", "type": "string", "__id__": "Productid"},
                {"name": "Category", "type": "string", "__id__": "Category"},
                {"name": "Subcategory", "type": "string", "__id__": "Subcategory"},
                {"name": "Productname", "type": "string", "__id__": "Productname"},
                {"name": "Sales", "type": "number", "__id__": "Sales"},
                {"name": "Quantity", "type": "number", "__id__": "Quantity"},
                {"name": "Discount", "type": "number", "__id__": "Discount"},
                {"name": "Profit", "type": "number", "__id__": "Profit"},
                {"name": "Month", "type": "string", "__id__": "Month"},
                {"name": "MonthYear", "type": "datetime", "__id__": "Monthyear"}
            ]
            if x["name"] in fields
        ],
    }

In [ ]:
import altair as alt

alt.data_transformers.disable_max_rows()
show(
    data(["Category","Sales"]),
    {
        "view": [
            {
                "coordinates": "cartesian",
                "mark": [
                    {
                        "type": "bar",
                        "encoding": [
                            {"channel": "x", "field": "Category"},
                            {"channel": "y", "field": "Sales"}
                            ],
                    }
                ],
                "scale": [
                    {"channel": "x", "type": "ordinal"},
                    {"channel": "y", "type": "linear", "zero": "true"}
                ],
            }
        ]
    },
)

In [ ]:
alt.data_transformers.disable_max_rows()
show(
    data(["Category","Subcategory","Sales"]),
    {
        "view": [
            {
                "mark": [
                    {
                        "type": "bar",
                        "encoding": [
                            {"channel": "x", "field": "Subcategory"},
                            {"channel": "y", "field": "Sales"},
                        ],
                    }
                ],
                "scale": [
                    {"channel": "x", "type": "ordinal"},
                    {"channel": "y", "type": "linear", "zero": "true"},
                ],
                "facet": [
                    {"channel": "col", "field": "Category"},
                ],
            }
        ]
    },
)

# Relationship [R1]

## -- Interpretação --

- Vendas (Métrica)
- Soma (Agregação)
- Ciclicos (Tempo)

In [ ]:
import altair as alt

alt.data_transformers.disable_max_rows()
show(
    data(["Sales", "Month"]),
    {
        "view": [
            {
                "mark": [
                    {
                        "type": "bar",
                        "encoding": [
                            {"channel": "x", "field": "Month"},
                            {"channel": "y", "field": "Sales"},
                        ],
                    }
                ],
                "scale": [
                    {"channel": "x", "type": "ordinal"},
                    {"channel": "y", "type": "linear", "zero": "true"},
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Sales", "Month"]),
    {
        "view": [
            {
                "mark": [
                    {
                        "type": "bar",
                        "encoding": [
                            {"channel": "x", "field": "Month"},
                            {"channel": "y", "field": "Sales", "aggregate": "sum"},
                        ],
                    }
                ],
                "scale": [
                    {"channel": "x", "type": "ordinal"},
                    {"channel": "y", "type": "linear", "zero": "true"},
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Sales", "Month"]),
    {
        "view": [
            {
                "coordinates": "polar",
                "mark": [
                    {
                        "type": "bar",
                        "encoding": [
                            {
                                "channel": "y",
                                "field": "Sales",
                                "aggregate": "sum",
                                "stack": "zero",
                            },
                            {"channel": "color", "field": "Month"},
                        ],
                    }
                ],
                "scale": [
                    {"channel": "y", "type": "linear", "zero": "true"},
                    {"channel": "color", "type": "categorical"},
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Sales", "Month"]),
    {
        "view": [
            {
                "coordinates": "polar",
                "mark": [
                    {
                        "type": "bar",
                        "encoding": [
                            {"channel": "x", "field": "Month"},
                            {
                                "channel": "y",
                                "field": "Sales",
                                "aggregate": "sum",
                            },
                        ],
                    },
                ],
                "scale": [
                    {"channel": "x", "type": "ordinal"},
                    {"channel": "y", "type": "linear", "zero": "true"},
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Sales", "Month"]),
    {
        "view": [
            {
                "mark": [
                    {
                        "type": "bar",
                        "encoding": [
                            {"channel": "y", "aggregate": "sum", "field": "Sales","stack": "zero"},
                            {"channel": "color", "field": "Month"},
                        ],
                    }
                ],
                "scale": [
                    {
                        "channel": "y",
                        "type": "linear",
                        "zero": "true",
                    },
                    {"channel": "color", "type": "categorical"},
                ],
            }
        ]
    },
)

# Relationship [R1]

## -- Interpretação --

- Vendas (Métrica)
- Média (Agregação)
- Ciclicos (Tempo)

In [ ]:
show(
    data(["Sales", "Month"]),
    {
        "view": [
            {
                "mark": [
                    {
                        "type": "text",
                        "encoding": [
                            {"channel": "x", "field": "Month"},
                            {"channel": "y", "field": "Sales", "aggregate": "mean"},
                        ],
                    }
                ],
                "scale": [
                    {"channel": "x", "type": "ordinal"},
                    {"channel": "y", "type": "linear", "zero": "true"},
                ],
            }
        ]
    },
)

# Relationship [R1]

## -- Interpretação --

- Vendas (Métrica)
- Média (Agregação)
- Linear (Tempo)

In [ ]:
show(
    data(["Sales", "MonthYear"]),
    {
        "view": [
            {
                "mark": [
                    {
                        "type": "bar",
                        "encoding": [
                            {"channel": "x", "field": "MonthYear"},
                            {"channel": "y", "field": "Sales", "aggregate": "mean"},
                        ],
                    }
                ],
                "scale": [
                    {"channel": "x", "type": "ordinal"},
                    {"channel": "y", "type": "linear", "zero": "true"},
                ],
            }
        ]
    },
)

## -- Interpretação --

- Vendas (Métrica)
- Soma (Agregação)
- Linear (Tempo)

In [ ]:
show(
    data(["Sales", "MonthYear"]),
    {
        "view": [
            {
                "mark": [
                    {
                        "type": "bar",
                        "encoding": [
                            {"channel": "x", "field": "MonthYear"},
                            {"channel": "y", "field": "Sales", "aggregate": "sum"},
                        ],
                    }
                ],
                "scale": [
                    {"channel": "x", "type": "ordinal"},
                    {"channel": "y", "type": "linear", "zero": "true"},
                ],
            }
        ]
    },
)

In [ ]:
import altair as alt

alt.data_transformers.disable_max_rows()
show(
    data(["Sales", "MonthYear"]),
    {
        "view": [
            {
                "mark": [
                     {
                        "type": "bar",
                        "encoding": [{"channel": "x", "field": "MonthYear"}, {"channel": "y", "aggregate": "sum", "field": "Sales"}]
                    },
                    {
                        "type": "text",
                        "encoding": [
                            {"channel": "text", "aggregate": "sum", "field": "Sales"},
                        ],
                    },
                ],
                "scale": [
                    {"channel": "text", "type": "ordinal"},
                    {"channel": "y", "type": "linear", "zero": "true"},
                    {"channel": "x", "type": "ordinal"}
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Sales", "MonthYear"]),
    {
        "view": [
            {
                "mark": [
                     {
                        "type": "bar",
                        "encoding": [{"channel": "x", "aggregate": "sum", "field": "Sales"}, {"channel": "y", "field": "MonthYear"}]
                    },
                    {
                        "type": "text",
                        "encoding": [
                            {"channel": "text", "aggregate": "sum", "field": "Sales"},
                        ],
                    },
                ],
                "scale": [
                    {"channel": "text", "type": "ordinal"},
                    {"channel": "x", "type": "linear", "zero": "true"},
                    {"channel": "y", "type": "ordinal"}
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Sales", "MonthYear"]),
    {
        "view": [
            {
                "mark": [
                    {
                        "type": "bar",
                        "encoding": [
                            {"channel": "y", "field": "MonthYear"},
                            {"channel": "x", "field": "Sales", "aggregate": "sum"},
                        ],
                    }
                ],
                "scale": [
                    {"channel": "y", "type": "ordinal"},
                    {"channel": "x", "type": "linear", "zero": "true"},
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Sales", "MonthYear"]),
    {
        "view": [
            {
                "mark": [
                     {
                        "type": "line",
                        "encoding": [{"channel": "x", "field": "MonthYear"}, {"channel": "y", "aggregate": "sum", "field": "Sales"}]
                    },
                    {
                        "type": "text",
                        "encoding": [
                            {"channel": "text", "aggregate": "sum", "field": "Sales"},
                        ],
                    },
                ],
                "scale": [
                    {"channel": "text", "type": "ordinal"},
                    {"channel": "y", "type": "linear", "zero": "true"},
                    {"channel": "x", "type": "ordinal"}
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Sales", "MonthYear"]),
    {
        "view": [
            {
                "mark": [
                     {
                        "type": "line",
                        "encoding": [{"channel": "x", "aggregate": "sum", "field": "Sales"}, {"channel": "y", "field": "MonthYear"}]
                    },
                    {
                        "type": "text",
                        "encoding": [
                            {"channel": "text", "aggregate": "sum", "field": "Sales"},
                        ],
                    },
                ],
                "scale": [
                    {"channel": "text", "type": "ordinal"},
                    {"channel": "x", "type": "linear", "zero": "true"},
                    {"channel": "y", "type": "ordinal"}
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Sales", "MonthYear"]),
    {
        "view": [
            {
                "mark": [
                     {
                        "type": "tick",
                        "encoding": [{"channel": "x", "field": "MonthYear"}, {"channel": "y", "aggregate": "sum", "field": "Sales"}]
                    },
                    {
                        "type": "text",
                        "encoding": [
                            {"channel": "text", "aggregate": "sum", "field": "Sales"},
                        ],
                    },
                ],
                "scale": [
                    {"channel": "text", "type": "ordinal"},
                    {"channel": "y", "type": "linear", "zero": "true"},
                    {"channel": "x", "type": "ordinal"}
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Sales", "MonthYear"]),
    {
        "view": [
            {
                "mark": [
                     {
                        "type": "tick",
                        "encoding": [{"channel": "x", "aggregate": "sum", "field": "Sales"}, {"channel": "y", "field": "MonthYear"}]
                    },
                    {
                        "type": "text",
                        "encoding": [
                            {"channel": "text", "aggregate": "sum", "field": "Sales"},
                        ],
                    },
                ],
                "scale": [
                    {"channel": "text", "type": "ordinal"},
                    {"channel": "x", "type": "linear", "zero": "true"},
                    {"channel": "y", "type": "ordinal"}
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Sales", "MonthYear"]),
    {
        "view": [
            {
                "mark": [
                     {
                        "type": "point",
                        "encoding": [{"channel": "x", "aggregate": "sum", "field": "Sales"}, {"channel": "y", "field": "MonthYear"}]
                    },
                    {
                        "type": "text",
                        "encoding": [
                            {"channel": "text", "aggregate": "sum", "field": "Sales"},
                        ],
                    },
                ],
                "scale": [
                    {"channel": "text", "type": "ordinal"},
                    {"channel": "x", "type": "linear", "zero": "true"},
                    {"channel": "y", "type": "ordinal"}
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Sales", "MonthYear"]),
    {
        "view": [
            {
                "mark": [
                     {
                        "type": "point",
                        "encoding": [{"channel": "x", "field": "MonthYear"}, {"channel": "y", "aggregate": "sum", "field": "Sales"}]
                    },
                    {
                        "type": "text",
                        "encoding": [
                            {"channel": "text", "aggregate": "sum", "field": "Sales"},
                        ],
                    },
                ],
                "scale": [
                    {"channel": "text", "type": "ordinal"},
                    {"channel": "y", "type": "linear", "zero": "true"},
                    {"channel": "x", "type": "ordinal"}
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Sales", "MonthYear"]),
    {
        "view": [
            {
                "mark": [
                    {
                        "type": "point",
                        "encoding": [
                            {"channel": "x", "field": "MonthYear"},
                            {"channel": "y", "field": "Sales", "aggregate": "sum"},
                        ],
                    }
                ],
                "scale": [
                    {"channel": "x", "type": "ordinal"},
                    {"channel": "y", "type": "linear", "zero": "true"},
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Sales", "MonthYear"]),
    {
        "view": [
            {
                "mark": [
                    {
                        "type": "point",
                        "encoding": [
                            {"channel": "x", "field": "MonthYear"},
                            {"channel": "y", "field": "Sales", "aggregate": "sum"},
                            {"channel": "size", "field": "Sales", "aggregate": "sum"}
                        ],
                    }
                ],
                "scale": [
                    {"channel": "x", "type": "ordinal"},
                    {"channel": "y", "type": "linear", "zero": "true"},
                    {"channel": "size", "type": "linear"}
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Sales", "MonthYear"]),
    {
        "view": [
            {
                "mark": [
                     {
                        "type": "area",
                        "encoding": [{"channel": "x", "field": "MonthYear"}, {"channel": "y", "aggregate": "sum", "field": "Sales"}]
                    },
                    {
                        "type": "text",
                        "encoding": [
                            {"channel": "text", "aggregate": "sum", "field": "Sales"},
                        ],
                    },
                ],
                "scale": [
                    {"channel": "text", "type": "ordinal"},
                    {"channel": "y", "type": "linear", "zero": "true"},
                    {"channel": "x", "type": "ordinal"}
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Sales", "MonthYear"]),
    {
        "view": [
            {
                "mark": [
                    {
                        "type": "area",
                        "encoding": [
                            {"channel": "x", "field": "MonthYear"},
                            {"channel": "y", "field": "Sales", "aggregate": "sum"},
                        ],
                    }
                ],
                "scale": [
                    {"channel": "x", "type": "ordinal"},
                    {"channel": "y", "type": "linear", "zero": "true"},
                ],
            }
        ]
    },
)

In [ ]:
show(
    data(["Sales", "Month", "Profit"]),
    {
        "view": [
            {
                "mark": [
                    {
                        "type": "rect",
                        "encoding": [
                            {"channel": "x", "field": "Profit", "binning": 10},
                            {"channel": "y", "field": "Month"},
                            {"channel": "color", "field": "Sales", "aggregate": "sum"}
                        ],
                    }
                ],
                "scale": [
                    {"channel": "x", "type": "ordinal"},
                    {"channel": "y", "type": "ordinal"},
                    {"channel": "color", "type": "linear"}
                ],
            }
        ]
    },
)